## Visualize subcortical pRF data for manuscript     
First set up some paths and variables

In [1]:
ref_path=/home/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/Reference
res_path=/home/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/MRI

declare -a subs=(danny eddy)

for SUBJ in ${subs[@]}; do
    eval ${SUBJ}_t1=${ref_path}/Volumes/anat/${SUBJ}/sub-${SUBJ}_ref_anat_brain_res-0.5x0.5x0.5.nii.gz
    eval ${SUBJ}_d99=${ref_path}/Volumes/atlas/${SUBJ}/D99_in_${SUBJ^}_adj.nii.gz
    eval ${SUBJ}_roi_path=${ref_path}/Volumes/atlas/${SUBJ}/ROI_manualadjust/1mm
done

Where to put output

In [2]:
subcort_path=${res_path}/subcort
mkdir -p ${subcort_path}

Create some subcortical ROI masks

In [18]:
# cat rois
for SUBJ in ${subs[@]}; do
    eval fslmaths '$'${SUBJ}_roi_path/LGNm.nii.gz -add '$'${SUBJ}_roi_path/LGNp.nii.gz -bin ${subcort_path}/${SUBJ}_LGN.nii.gz
    eval fslmaths '$'${SUBJ}_roi_path/PL.nii.gz -add '$'${SUBJ}_roi_path/PM.nii.gz -bin ${subcort_path}/${SUBJ}_PULV.nii.gz
    eval fslmaths ${subcort_path}/${SUBJ}_PULV.nii.gz -add ${subcort_path}/${SUBJ}_LGN.nii.gz -bin ${subcort_path}/${SUBJ}_LGNPULV.nii.gz
    eval cp '$'${SUBJ}_roi_path/Striatum.nii.gz ${subcort_path}/${SUBJ}_Striatum.nii.gz
done

## Which models / HRF / R2-threshold

In [3]:
model=csshrf
hrf=mhrf
RTH=2

## Show suprathreshold angles / eccentricity / size

In [4]:
M=${model}_cv1_${hrf}
#M=${model}_cv1_${hrf}_neggain

for SUBJ in ${subs[@]}; do
    eval ${SUBJ}_R2=${res_path}/${SUBJ}/${M}/inAnat/Sess-${model}_cv1_${hrf}_meanR2.nii.gz
    eval ${SUBJ}_ANG=${res_path}/${SUBJ}/${M}/inAnat/TH_$RTH/ANG_th${RTH}.nii.gz
    eval ${SUBJ}_ECC=${res_path}/${SUBJ}/${M}/inAnat/TH_$RTH/ECC_th${RTH}.nii.gz
    eval ${SUBJ}_X=${res_path}/${SUBJ}/${M}/inAnat/TH_$RTH/X_th${RTH}.nii.gz
    eval ${SUBJ}_Y=${res_path}/${SUBJ}/${M}/inAnat/TH_$RTH/Y_th${RTH}.nii.gz
    eval ${SUBJ}_RFS=${res_path}/${SUBJ}/${M}/inAnat/TH_$RTH/RFS_th${RTH}.nii.gz
done

In [57]:
P=${subcort_path}/${model}_${hrf}_${RTH}
#P=${subcort_path}/${model}_neggain_${hrf}_${RTH}
mkdir -p ${P}
for SUBJ in ${subs[@]}; do
    eval fslmaths '$'${SUBJ}_ANG -mas ${subcort_path}/${SUBJ}_LGN.nii.gz ${P}/${SUBJ}_ANG_LGN.nii.gz
    eval fslmaths '$'${SUBJ}_ANG -mas ${subcort_path}/${SUBJ}_PULV.nii.gz ${P}/${SUBJ}_ANG_PULV.nii.gz
    eval fslmaths '$'${SUBJ}_ANG -mas ${subcort_path}/${SUBJ}_Striatum.nii.gz ${P}/${SUBJ}_ANG_Striatum.nii.gz

    eval fslmaths '$'${SUBJ}_ECC -mas ${subcort_path}/${SUBJ}_LGN.nii.gz ${P}/${SUBJ}_ECC_LGN.nii.gz
    eval fslmaths '$'${SUBJ}_ECC -mas ${subcort_path}/${SUBJ}_PULV.nii.gz ${P}/${SUBJ}_ECC_PULV.nii.gz
    eval fslmaths '$'${SUBJ}_ECC -mas ${subcort_path}/${SUBJ}_Striatum.nii.gz ${P}/${SUBJ}_ECC_Striatum.nii.gz
    
    eval fslmaths '$'${SUBJ}_X -mas ${subcort_path}/${SUBJ}_LGN.nii.gz ${P}/${SUBJ}_X_LGN.nii.gz
    eval fslmaths '$'${SUBJ}_X -mas ${subcort_path}/${SUBJ}_PULV.nii.gz ${P}/${SUBJ}_X_PULV.nii.gz
    eval fslmaths '$'${SUBJ}_X -mas ${subcort_path}/${SUBJ}_Striatum.nii.gz ${P}/${SUBJ}_X_Striatum.nii.gz

    eval fslmaths '$'${SUBJ}_Y -mas ${subcort_path}/${SUBJ}_LGN.nii.gz ${P}/${SUBJ}_Y_LGN.nii.gz
    eval fslmaths '$'${SUBJ}_Y -mas ${subcort_path}/${SUBJ}_PULV.nii.gz ${P}/${SUBJ}_Y_PULV.nii.gz
    eval fslmaths '$'${SUBJ}_Y -mas ${subcort_path}/${SUBJ}_Striatum.nii.gz ${P}/${SUBJ}_Y_Striatum.nii.gz

    eval fslmaths '$'${SUBJ}_RFS -mas ${subcort_path}/${SUBJ}_LGN.nii.gz ${P}/${SUBJ}_RFS_LGN.nii.gz
    eval fslmaths '$'${SUBJ}_RFS -mas ${subcort_path}/${SUBJ}_PULV.nii.gz ${P}/${SUBJ}_RFS_PULV.nii.gz
    eval fslmaths '$'${SUBJ}_RFS -mas ${subcort_path}/${SUBJ}_Striatum.nii.gz ${P}/${SUBJ}_RFS_Striatum.nii.gz
done

In [5]:
for SUBJ in ${subs[@]}; do
    M=${subcort_path}/${model}_${hrf}_${RTH}
    P=${subcort_path}/${model}_${hrf}_${RTH}/${SUBJ}
    eval '/opt/FSLeyes/fsleyes -no $'${SUBJ}_t1 \
        '$'${SUBJ}_R2 -dr 0 5 -cm brain_colours_5redyell_iso \
        ${P}_ANG_LGN.nii.gz -dr 0 360 -cm hsv \
        ${P}_ANG_PULV.nii.gz -dr 0 360 -cm hsv \
        ${P}_ANG_Striatum.nii.gz -dr 0 360 -cm hsv \
        ${P}_ECC_LGN.nii.gz -dr 0 10 -cm brain_colours_5redyell_iso \
        ${P}_ECC_PULV.nii.gz -dr 0 10 -cm brain_colours_5redyell_iso \
        ${P}_ECC_Striatum.nii.gz -dr 0 10 -cm brain_colours_5redyell_iso \
        ${P}_RFS_LGN.nii.gz -dr 0 4 -cm brain_colours_5redyell_iso \
        ${P}_RFS_PULV.nii.gz -dr 0 4 -cm brain_colours_5redyell_iso \
        ${P}_RFS_Striatum.nii.gz -dr 0 4 -cm brain_colours_5redyell_iso &       
done

#         ${P}_X_LGN.nii.gz -dr -10 10 -cm render3 \
#         ${P}_X_PULV.nii.gz -dr -10 10 -cm render3 \
#         ${P}_X_Striatum.nii.gz -dr -10 10 -cm render3 \
#         ${P}_Y_LGN.nii.gz -dr -10 10 -cm render3 \
#         ${P}_Y_PULV.nii.gz -dr -10 10 -cm render3 \
#         ${P}_Y_Striatum.nii.gz -dr -10 10 -cm render3 \

[1] 4521
[2] 4522


In [74]:
eval echo '$'${SUBJ}_R2
eval echo ${P}_ANG_LGN.nii.gz


(fsleyes:11686): dbind-WARNING **: 12:53:20.873: Couldn't register with accessibility bus: Did not receive a reply. Possible causes include: the remote application did not send a reply, the message bus security policy blocked the reply, the reply timeout expired, or the network connection was broken.

(fsleyes:11685): dbind-WARNING **: 12:53:20.886: Couldn't register with accessibility bus: Did not receive a reply. Possible causes include: the remote application did not send a reply, the message bus security policy blocked the reply, the reply timeout expired, or the network connection was broken.
unrecognized arguments: /home/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/MRI/eddy/css_cv1_mhrf/inAnat/Sess-css_cv1_mhrf_meanR2.nii.gz /home/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/MRI/subcort/css_mhrf_2/eddy_ANG_LGN.nii.gz /home/chris/Documents/CURRENT_PROJECTS/NHP_MRI/Projects/pRF/FitResults/MRI/subcort/css_mhrf_2/eddy_ANG_PULV.nii.gz /home/chr